### Dependencies

In [1]:
%pip install --upgrade pip
%pip install selenium
%pip install beautifulsoup4
%pip install pandas

  Using cached pip-23.1.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
  Rolling back uninstall of pip
  Moving to c:\users\ccamp\appdata\roaming\python\python310\scripts\pip.exe
   from C:\Users\ccamp\AppData\Local\Temp\pip-uninstall-xm9zk8du\pip.exe
  Moving to c:\users\ccamp\appdata\roaming\python\python310\scripts\pip3.10.exe
   from C:\Users\ccamp\AppData\Local\Temp\pip-uninstall-xm9zk8du\pip3.10.exe
  Moving to c:\users\ccamp\appdata\roaming\python\python310\scripts\pip3.exe
   from C:\Users\ccamp\AppData\Local\Temp\pip-uninstall-xm9zk8du\pip3.exe
  Moving to c:\users\ccamp\appdata\roaming\python\python310\site-packages\pip-23.0.1.dist-info\
Note: you may need to restart the kernel to use updated packages.   from C:\Users\ccamp\AppData\Roaming\Python\Python310\site-packages\~ip-23.0.1.dist-info


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\ccamp\AppData\Roaming\Python\Python310\site-packages\pip\_internal\cli\base_command.py", line 160, in exc_logging_wrapper
    status = run_func(*args)
  File "C:\Users\ccamp\AppData\Roaming\Python\Python310\site-packages\pip\_internal\cli\req_command.py", line 247, in wrapper
    return func(self, options, args)
  File "C:\Users\ccamp\AppData\Roaming\Python\Python310\site-packages\pip\_internal\commands\install.py", line 507, in run
    installed = install_given_reqs(
  File "C:\Users\ccamp\AppData\Roaming\Python\Python310\site-packages\pip\_internal\req\__init__.py", line 73, in install_given_reqs
    requirement.install(
  File "C:\Users\ccamp\AppData\Roaming\Python\Python310\site-packages\pip\_internal\req\req_install.py", line 796, in install
    install_wheel(
  File "C:\Users\ccamp\AppData\Roaming\Python\Python310\site-packages\pip\_internal\operations\install\wheel.py", line 729, in install_wheel
    _install_


  Moving to c:\users\ccamp\appdata\roaming\python\python310\site-packages\pip\
   from C:\Users\ccamp\AppData\Roaming\Python\Python310\site-packages\~ip



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:

from bs4 import BeautifulSoup,ResultSet
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib.parse import urlparse, parse_qs
from pprint import pprint
import pandas as pd
from pandas import DataFrame
import time
import datetime

### driver
source has a dynamic data so for avoid empty tables is required use a selenium web driver.

In [3]:
driver = webdriver.Edge()

# Auxiliary functions

In [4]:
#request dynamic url


url = lambda page,sector : f'https://seia.sea.gob.cl/busqueda/buscarProyectoAction.php?nombre=&sector={sector}&_paginador_refresh=1&_paginador_fila_actual={page}'
pages_quantity = lambda to_page: range(1,to_page+1)

#table extraction for each individual page data
def get_DataFrame(page_source:str)->DataFrame:
    soup = BeautifulSoup(page_source)
    table = soup.find('tbody')
    rows = table.find_all('tr')
    
    print(f"got {len(rows)} rows of data")
    
    #extract data from each row
    extract_table = list(map(lambda row: get_row_data(row),rows))
    df = pd.DataFrame(extract_table)
    return df
    

#row extraction to formatted object    
def get_row_data(row:ResultSet[any])->dict[str,str|int|float]:
    data = row.find_all('td')
    
    #extraction
    dossier_url = data[1].find('a',href=True)['href']
    dossier_id:int = int(parse_qs(urlparse(dossier_url).query)['id_expediente'][0])
    
    #object build
    data_built = {
        "corr": int(data[0].text),
        "dossier_id":dossier_id,
        "project":data[1].text,
        # "dossier_url":dossier_url,
        "instrument":data[2].text,
        "zone":data[3].text,
        "typology":data[4].text,
        "owner":data[5].text.strip().lower(),
        "inverts MM$USD":float(data[6].text.replace(".","").replace(",",".")),
        "date_create":datetime.datetime.strptime(data[7].text,"%d/%m/%Y"),
        "dossier_status":data[8].text
    }
    return data_built
    

### Loop

In [6]:
PAGES_TO_SCRAP:int = 100
list_DataFrames:list[DataFrame] = []
for page in pages_quantity(PAGES_TO_SCRAP):
    #get page from selenium
    print(f"getting data from page {page} of {PAGES_TO_SCRAP}")
    driver.get(url(page,7))
    page_source:str = driver.page_source
    list_DataFrames.append(get_DataFrame(page_source))
    
result = pd.concat(list_DataFrames)

getting data from page 1 of 100
got 10 rows of data
getting data from page 2 of 100
got 10 rows of data
getting data from page 3 of 100
got 10 rows of data
getting data from page 4 of 100
got 10 rows of data
getting data from page 5 of 100
got 10 rows of data
getting data from page 6 of 100
got 10 rows of data
getting data from page 7 of 100
got 10 rows of data
getting data from page 8 of 100
got 10 rows of data
getting data from page 9 of 100
got 10 rows of data
getting data from page 10 of 100
got 10 rows of data
getting data from page 11 of 100
got 10 rows of data
getting data from page 12 of 100
got 10 rows of data
getting data from page 13 of 100
got 10 rows of data
getting data from page 14 of 100
got 10 rows of data
getting data from page 15 of 100
got 10 rows of data
getting data from page 16 of 100
got 10 rows of data
getting data from page 17 of 100
got 10 rows of data
getting data from page 18 of 100
got 10 rows of data
getting data from page 19 of 100
got 10 rows of data
ge

In [8]:
%pip install openpyxl
import openpyxl
result.to_excel('out.xlsx')
print("done")

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


done
